# Mutação

## Definição de classes

### População

In [1]:
from numpy.random import randint
from numpy import argsort, unique


class Populacao:
    """
    Cria e avalia uma população

    Attributes
    ----------
    func_avaliacao: func
        Função que recebe um indivívudo como entrada e retorna um valor numérico
    total_genes: int
        Número inteiro que representa o tamanho da cadeia
    total_populacao: int
        Numéro inteiro que representa o número total de indivíduos na população

    Methods
    -------
    gerar_populacao: None
        Gera uma população aleatória
    avaliar: numpy.array
        Retorna os valores da população ordernados
    """

    def __init__(self, func_avaliacao, total_genes, total_populacao):
        self.func_avaliacao = func_avaliacao
        self.total_genes = total_genes
        self.total_populacao = total_populacao
        self.populacao = None

    def gerar_populacao(self):
        """Gera uma população aleatória"""

        self.populacao = randint(0, 2, size=(self.total_populacao,
                                             self.total_genes),
                                 dtype='b')

    def avaliar(self):
        """
        Avalia e ordena a população

        Returns
        -------
        numpy.array: Os valores da população ordenados
        """
        u, indices = unique(self.populacao,
                            return_inverse=True,
                            axis=0)

        valores = self.func_avaliacao(u)

        valores = valores[indices]

        ind = argsort(valores)

        self.populacao[:] = self.populacao[ind]

        return valores[ind]


### Seleção

#### Classe Base

In [2]:
from numpy import array

class SelecaoBase:

    def __init__(self, populacao):
        self.populacao = populacao

    def selecionar(self, fitness):
        """
        Retorna lista de indices do vetor da populacao dos
        individuos selecionados
        """
        raise NotImplementedError('Ainda não implementado')

    def selecao(self, n, fitness=None):
        progenitores = array([self.selecionar(fitness)
                              for _ in range(n)])
        return self.populacao.populacao[progenitores]

#### Roleta

In [3]:
from numpy import array
from numpy.random import random

class Roleta(SelecaoBase):
    """
    Seleção por meio do método de Roleta
    """

    def __init__(self, populacao):
        super(Roleta, self).__init__(populacao)

    def selecionar(self, fitness):
        if fitness is None:
            fitness = self.populacao.avaliar()

        fitness_min = fitness.min()

        fitness = fitness - fitness_min

        total = fitness.sum()

        parada = total * (1.0 - random())

        parcial, i = 0, 0

        while True:
            if i > fitness.size - 1:
                break

            parcial += fitness[i]

            if parcial >= parada:
                break

            i += 1

        return i - i


#### Classificação

In [4]:
from numpy import array
from numpy import argsort
from numpy.random import random

class Classificacao(SelecaoBase):
    
    def __init__(self, populacao):
        super(Classificacao, self).__init__(populacao)
        
    def selecionar(self, fitness):
        if fitness is None:
            fitness = self.populacao.avaliar()
            
        classificacao = argsort(fitness) + 1

        total = classificacao.sum()
        
        parada = total * random()
        
        parcial, i = 0, 0
        
        while True:
            if i > fitness.size - 1:
                break
            
            parcial += classificacao[i]
            
            if parcial >= parada:
                break
            
            i += 1
            
        return i + 1

#### Torneio

In [5]:
from numpy import array
from numpy import where
from numpy.random import choice

class Torneio(SelecaoBase):
    
    def __init__(self, populacao, tamanho=10):
        super(Torneio, self).__init__(populacao)
        self.tamanho = tamanho
        
    def selecionar(self, fitness):
        if fitness is None:
            fitness = self.populacao.avaliar()
        
        grupo = choice(fitness, size=self.tamanho)
        
        campeao = grupo.max()
        
        i = where(fitness == campeao)[0][0]
        
        return i

### Cruzamento

#### Classe abstrata

In [6]:
from numpy import array
from numpy.random import random
from numpy.random import randint

class TamanhoIndividuosIncompativelError(Exception):
    pass

class CruzamentoBase:
    
    def __init__(self, total_populacao):
        self.total_populacao = total_populacao
        
    def cruzamento(self, progenitor_1, progenitor_2):
        raise NotImplementedError('Precisa ser implementado')
    
    def descendentes(self, subpopulacao, cruzamento_proba):
        """
        Retorna uma nova população do tamanho informado através do cruzamento
        """
        nova_populacao = []
        
        n_pop = len(subpopulacao)
        
        while len(nova_populacao) < self.total_populacao:
            i, j = randint(0, n_pop - 1), randint(0, n_pop - 1)
            
            while j == i:
                j = randint(0, n_pop - 1)
                
            cruzar = random()
            
            if cruzar < cruzamento_proba:
                desc_1, desc_2 = self.cruzamento(subpopulacao[i],
                                                 subpopulacao[j])
                
                nova_populacao.append(desc_1)
                
                if len(nova_populacao) < self.total_populacao:
                    nova_populacao.append(desc_2)
        
        return array(nova_populacao)

#### Ponto

In [7]:
from numpy import array
from numpy.random import randint

class Ponto(CruzamentoBase):
    
    def __init__(self, total_populacao):
        super(Ponto, self).__init__(total_populacao)
        
    def cruzamento(self, progenitor_1, progenitor_2):
        n_1, n_2 = len(progenitor_1), len(progenitor_2)
        
        if n_1 != n_2:
            exp_msg = f'Tamanhos não compatíveis {n_1} != {n_2}'
            raise TamanhoIndividuosIncompativelError(exp_msg)

        ponto = randint(1, n_1 - 1)
        
        desc_1, desc_2 = progenitor_1.copy(), progenitor_2.copy()
        
        desc_1[ponto:], desc_2[ponto:] = progenitor_2[ponto:], progenitor_1[ponto:]
        
        return desc_1, desc_2

#### k-Pontos

In [8]:
from numpy import array
from numpy.random import random
from numpy.random import randint

class KPontos(CruzamentoBase):
    
    def __init__(self, total_populacao):
        super(KPontos, self).__init__(total_populacao)
        
    def cruzamento(self, progenitor_1, progenitor_2):
        n_1, n_2 = len(progenitor_1), len(progenitor_2)
        
        if n_1 != n_2:
            exp_msg = f'Tamanhos não compatíveis {n_1} != {n_2}'
            raise TamanhoIndividuosIncompativelError(exp_msg)
        
        desc_1, desc_2 = progenitor_1.copy(), progenitor_2.copy()
        
        k_pontos = randint(1, n_1 - 2)
        
        k_pontos_lista = []
        
        while len(k_pontos_lista) <= k_pontos:
            ponto = randint(1, n_1 - 1)
            
            if ponto not in k_pontos_lista:
                k_pontos_lista.append(ponto)
                
        k_pontos_lista.sort()
        
        troca = randint(0, 1)
        
        for i in range(k_pontos):
            if troca == 0:
                troca = 1
            else:
                troca = 0
                desc_1[k_pontos_lista[i]:k_pontos_lista[i + 1]] = progenitor_2[k_pontos_lista[i]:k_pontos_lista[i + 1]]
                desc_2[k_pontos_lista[i]:k_pontos_lista[i + 1]] = progenitor_1[k_pontos_lista[i]:k_pontos_lista[i + 1]]
        
        return desc_1, desc_2

#### Embaralhamento

In [9]:
from numpy.random import randint
from numpy.random import shuffle    

class Embaralhamento(CruzamentoBase):
    
    def __init__(self, total_populacao):
        super(Embaralhamento, self).__init__(total_populacao)
        
    def cruzamento(self, progenitor_1, progenitor_2):
        n_1, n_2 = len(progenitor_1), len(progenitor_2)
        
        if n_1 != n_2:
            exp_msg = f'Tamanhos não compatíveis {n_1} != {n_2}'
            raise TamanhoIndividuosIncompativelError(exp_msg)
        
        order = list(range(n_1))
        
        shuffle(order)
        
        ponto = randint(1, n_1 - 1)
        
        desc_1, desc_2 = progenitor_1.copy(), progenitor_2.copy()
        
        desc_1[:], desc_2[:] = desc_1[order], desc_2[order]
        
        desc_1[ponto:], desc_2[ponto:] = desc_2[ponto:], desc_1[ponto:]
        
        tmp_1, tmp_2 = desc_1.copy(), desc_2.copy()
        
        for i, j in enumerate(order):
            desc_1[j], desc_2[j] = tmp_1[i], tmp_2[i]
        
        return desc_1, desc_2

### Mutação

#### Classe abstrata

In [10]:
from numpy import array
from numpy.random import random
from numpy.random import randint

class MutacaoBase:
    
    def __init__(self, mutacao_proba):
        self.mutacao_proba = mutacao_proba
        self._populacao = None
        self.n_populacao = None
        self.n_genes = None
        
    def _set_populacao(self, populacao):
        self._populacao = populacao
        self.n_populacao = self._populacao.shape[0]
        self.n_genes = self._populacao.shape[1]
        
    def _get_populacao(self):
        return self._populacao
    
    def selecao(self):
        n_mutacoes = array([i for i in range(self.n_populacao)
                              if random() < self.mutacao_proba])
        return n_mutacoes

    def mutacao(self):
        raise NotImplementedError('Precisa ser implementado')
    
    populacao = property(_get_populacao, _set_populacao)

#### Flip

In [11]:
from numpy import array
from numpy.random import randint

class Flip(MutacaoBase):
    
    def __init__(self, mutacao_proba):
        super(Flip, self).__init__(mutacao_proba)
        
    def mutacao(self):
        n_mutacoes = self.selecao()
        
        genes_flip = array([randint(0, self.n_genes - 1) for _ in n_mutacoes])
        
        self.populacao[n_mutacoes, genes_flip] = 1 - self.populacao[n_mutacoes, genes_flip]

#### Dupla troca

In [12]:
from numpy import array

class DuplaTroca(MutacaoBase):
    
    def __init__(self, mutacao_proba):
        super(Flip, self).__init__(mutacao_proba)
        
    def mutacao(self):
        n_mutacoes = self.selecao()
        
        genes_1 = array([randint(0, self.n_genes - 1) for _ in n_mutacoes])
        genes_2 = array([randint(0, self.n_genes - 1) for _ in n_mutacoes])
        
        self.populacao[n_mutacoes, genes_1] = self.populacao[n_mutacoes, genes_2]
        self.populacao[n_mutacoes, genes_2] = self.populacao[n_mutacoes, genes_1]

#### Sequência reversa

In [14]:
from numpy import array
from numpy import newaxis
from numpy.random import randint

class SequenciaReversa(MutacaoBase):
    
    def __init__(self, mutacao_proba):
        super(Flip, self).__init__(mutacao_proba)
        
    def mutacao(self):
        n_mutacoes = self.selecao()
        
        if n_mutacoes.size != 0:
            
            for k in n_mutacoes:
                
                i, j = randint(0, self.n_genes - 1), randint(0, self.n_genes - 1)
                
                while i == j:
                    j = randint(0, self.n_genes - 1)
                    
                if i > j:
                    i, j = j, i
                    
                self.populacao[k, i:j] = self.populacao[k, i:j][::-1]

## Função de avaliação

### Função objetivo

$$
3(x - 1) ^ 2 exp(- (y + 1) ^ 2 - x ^ 2) - exp (- \frac{(x + 1)^2 - y^2}{3}) + exp (-x^2 -y^2) (10x^3 - 2x  + 10y^5)
$$

In [34]:
from numpy import exp

def func_objetivo(x, y):
    return 3 * exp(-(y + 1)**2 - x**2) \
           * (x - 1)**2 \
           - (exp(-(x + 1)**2 - y**2) / 3) \
           + exp(-x**2 -y**2) \
           * (10 * x**3 - 2 * x + 10 * y**5)

In [35]:
from numpy import exp

def bin(x):
    cnt = array([2 ** i for i in range(x.shape[1])])
    return array([(cnt * x[i,:]).sum() for i in range(x.shape[0])])

In [36]:
def xy(populacao):
    colunas = populacao.shape[1]

    meio = colunas // 2

    maior_bin = 2.0**meio - 1.0

    n_min, n_max = -3, 3

    const = (n_max - n_min) / maior_bin

    x = n_min + const * bin(populacao[:,:meio])
    y = n_min + const * bin(populacao[:,meio:])

    return x, y

In [37]:
def avaliacao(populacao):
    x, y = xy(populacao)

    tmp = func_objetivo(x, y)

    return tmp